## Reinforcement Learning 
For this project I will be using Q-learning, a form of temporal differencing to find the optimal path an agent should take to make it to an exit in a grid world.

In [82]:
# imports
import numpy as np

In [102]:
rows = 3
columns = 7
up = 0
down = 1
left = 2
right = 3
actions = [up,down,left,right]

# O is an open tile
# T is a treasure tile
# X is an obstacle tile
# E is an exit tile
# H is an enemy henchmen tile
# S is a start tile

# Gridworld

# OTXHXTE
# OOTOOOT
# STOHOOO

# agent can move up, down, left, and right within the grid
# there are 49 states/tiles
grid = [['O','T','X','X','X','T','E'],['O','O','T','O','O','O','T'],['S','T','O','H','X','O','O']]

#initialize a grid of reward/penalties corresponding to each grid position
# -1 penalty for moving to H tile
# +0.25 for moving to T tile
# +500 for moving to E tile
# If agent tries to move into an obstacle they will end up back in the tile they tried to move from. Hence no change
# wraps around like packman to the left and right but not up and down
i=0
reward = []
for r in range(rows):
    for c in range(columns):
        reward.append(list())
        for a in range(len(actions)):
            if a == up:
                row = r-1
                column = c
                if row<0 or column<0 or row>=rows or column>=columns:
                    reward[i].append(0)
                elif grid[row][column]=='O' or grid[row][column]=='X' or grid[row][column]=='S':
                    reward[i].append(0)
                elif grid[row][column]=='H':
                    reward[i].append(-1)
                elif grid[row][column]=='T':
                    reward[i].append(.25) 
                elif grid[row][column]=='E':
                    reward[i].append(500)
            elif a == down:
                row = r+1
                column = c
                if row<0 or column<0 or row>=rows or column>=columns:
                    reward[i].append(0)
                elif grid[row][column]=='O' or grid[row][column]=='X' or grid[row][column]=='S':
                    reward[i].append(0)
                elif grid[row][column]=='H':
                    reward[i].append(-1)
                elif grid[row][column]=='T':
                    reward[i].append(.25) 
                elif grid[row][column]=='E':
                    reward[i].append(500)
            elif a == left:
                row = r
                column = c-1
                if column < 0 and r>0:
                    row = r-1
                    column = columns-1
                if row<0 or column<0 or row>=rows or column>=columns:
                    reward[i].append(0)
                elif grid[row][column]=='O' or grid[row][column]=='X' or grid[row][column]=='S':
                    reward[i].append(0)
                elif grid[row][column]=='H':
                    reward[i].append(-1)
                elif grid[row][column]=='T':
                    reward[i].append(.25) 
                elif grid[row][column]=='E':
                    reward[i].append(500)
            elif a == right:
                row = r
                column = c+1
                if column > columns-1 and r<rows-1:
                    row = r+1
                    column = 0
                if row<0 or column<0 or row>=rows or column>=columns:
                    reward[i].append(0)
                elif grid[row][column]=='O' or grid[row][column]=='X' or grid[row][column]=='S':
                    reward[i].append(0)
                elif grid[row][column]=='H':
                    reward[i].append(-1)
                elif grid[row][column]=='T':
                    reward[i].append(.25) 
                elif grid[row][column]=='E':
                    reward[i].append(500)  
        i=i+1

print(reward)

# create Q-table that will be updated on each iteration
Q_table = np.zeros((len(reward),len(reward[0])))
print(Q_table)

# TODO make a penalty for each move

[[0, 0, 0, 0.25], [0, 0, 0, 0], [0, 0.25, 0.25, 0], [0, 0, 0, 0], [0, 0, 0, 0.25], [0, 0, 0, 500], [0, 0.25, 0.25, 0], [0, 0, 500, 0], [0.25, 0.25, 0, 0.25], [0, 0, 0, 0], [0, -1, 0.25, 0], [0, 0, 0, 0], [0.25, 0, 0, 0.25], [500, 0, 0, 0], [0, 0, 0.25, 0.25], [0, 0, 0, 0], [0.25, 0, 0.25, -1], [0, 0, 0, 0], [0, 0, -1, 0], [0, 0, 0, 0], [0.25, 0, 0, 0]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [103]:
def calcPosition(current_pos, action):
    if action == up:
        pos = current_pos-7
    elif action == down:
        pos = current_pos+7
    elif action == right:
        pos = current_pos+1
    else:
        pos = current_pos-1
    return pos


# Q-Learning alg
iterations = 10000
epsilon = 0.3
alpha = 0.05
gamma = .95
act =0
start_pos = 14
position = 14

for it in range(iterations):
    cont = True
    position = start_pos
    while cont:
        if epsilon > np.random.rand():
            act = np.random.choice(actions)
        else:
            act = np.argmax(reward[position]) 
        # TODO check if calc position is correct
        new_pos = calcPosition(position,act)
        # check for valid position
        r=new_pos//7
        c=new_pos%7
        if new_pos<0 or new_pos>=len(Q_table) or grid[r][c]=='X':
            new_pos = position
        r=new_pos//7
        c=new_pos%7
        # 0.2 penalty on each movement seen as subtracting from the reward
        Q_table[position][act] += alpha*(reward[position][act]-0.2+ gamma*np.max(Q_table[new_pos])-Q_table[position][act])
        position = new_pos
        if grid[r][c]=='E':
            cont = False
# round q_table
Q_table = np.array(Q_table)
Q_table = Q_table.round(1)    
print(Q_table)
    

[[450.7 474.6 450.7 428.2]
 [427.9 450.7 450.7 427.9]
 [  0.    0.    0.    0. ]
 [  0.    0.    0.    0. ]
 [  0.    0.    0.    0. ]
 [461.2 434.2 461.  499.8]
 [  0.    0.    0.    0. ]
 [450.2 450.6 499.8 449.1]
 [428.2 428.2 474.6 428.2]
 [427.9 406.6 450.7 406.6]
 [378.7 326.3 428.2 368.1]
 [417.8 378.  382.3 441.9]
 [474.9 415.3 386.5 468.8]
 [499.8 450.9 450.9 450.9]
 [474.6 450.9 474.9 428.2]
 [450.7 427.2 450.1 404.7]
 [428.2 353.9 403.4 331.7]
 [403.4 150.7 135.7 100.8]
 [  0.    0.    0.    0. ]
 [450.5 183.9 195.6 229.8]
 [474.9 420.  400.5 426.4]]


In [104]:
optimal_path = list()

exit = True
position = 14
while exit:
    direction = np.argmax(Q_table[position])
    
    if direction == up:
        optimal_path.append('Up')
    elif direction == down:
        optimal_path.append('Down')
    elif direction == left:
        optimal_path.append('Left')
    elif direction == right:
        optimal_path.append('Right')
    position = calcPosition(position,direction)
    if position == 6:
            break
print(optimal_path)
# Gridworld
# OTXHXTE
# OOTOOOT
# STOHOOO

['Left', 'Up']


This solution is in fact optimal. If the agent moves left, it will wrap around and end up a row higher at the rightmost column. There is a treasure in this cell. From there, if the agent moves up it will reach the exit. Due to the penalty for each move, this solution is optimal.